In [ ]:
''' 
Лабораторная работа №4. Различные подходы к решению задачи детектирования объектов
1. Загрузить набор данных «Номера тормозных башмаков» (data_sl.zip)
2. Провести предварительный анализ набора данных, провести оценку качества набора данных, указать возможные пути улучшения набора данных
3. Ознакомиться с материалами теории алгоритма фонтанного преобразования (например, https://newdisser.ru/_avtoreferats/01005395270.pdf, 
https://cyberleninka.ru/article/n/modelirovanie-protsessa-identifikatsii-graficheskihobektov/viewer)
4. Разработать модуль распознавания номера башмака при помощи фонтанного преобразования
5. Ознакомиться с материалами по архитектуре YOLOv11 (https://github.com/ultralytics/ultralytics, https://docs.ultralytics.com/quickstart/)
6. Загрузить предобученную модель YOLOv11 для решения задачи распознавания цифр как подвида задачи детектирования объектов/сегментации изображений
7. Разработать модуль распознавания номера башмака при помощи YOLOv11
8. Провести сравнительный анализ эффективности методов фонтанного преобразования и модели на архитектуре YOLOv11 для решения задачи распознавания 
номера тормозного башмака.
9. Сделать сравнительную таблицу с достоинствами, недостатками каждого алгоритма, особое внимание уделить точности и скорости работ
'''

In [1]:
#Фонтанное преобразование 1 датасет
import os
import cv2
import numpy as np
import pytesseract
import logging
from sklearn.metrics import precision_score, recall_score, average_precision_score

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Путь к исполняемому файлу Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Tesseract\tesseract.exe"
image_dir = 'Numbers_yolov11/train/images' # Путь к изображениям
label_dir = 'Numbers_yolov11/train/labels' # Путь к меткам изображений

def load_images_and_annotations(image_dir, label_dir):
    images = []
    annotations = []
    for filename in os.listdir(image_dir):
        if filename.endswith((".jpg", ".png")):
            # Загрузка изображения
            img_path = os.path.join(image_dir, filename)
            img = cv2.imread(img_path)
            images.append((filename, img))
            
            # Загрузка аннотации
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(label_dir, label_filename)
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    labels = [line.strip().split() for line in f.readlines()]
                annotations.append((filename, labels))
            else:
                annotations.append((filename, []))
    return images, annotations

def preprocess_image(img):
    # Преобразование в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Инвертирование цветов
    inverted = cv2.bitwise_not(gray)
    # Применение адаптивной бинаризации
    thresh = cv2.adaptiveThreshold(inverted, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)
    return thresh

def extract_digits(img):
    # Настройка конфигурации Tesseract для распознавания только цифр
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789'
    result = pytesseract.image_to_string(img, config=custom_config)
    return result.strip()

def calculate_metrics(true_labels, pred_labels):
    precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
    recall = recall_score(true_labels, pred_labels, average='macro', zero_division=0)
    average_precision = average_precision_score(true_labels, pred_labels, average='macro')
    return precision, recall, average_precision

#Загрузка изображений и меток
images, annotations = load_images_and_annotations(image_dir, label_dir)
all_true_labels = []
all_pred_labels = []
with open('results1.txt', 'w', encoding='utf-8') as f:
    for (filename, img), (_, labels) in zip(images, annotations):
        #print(f"Файл: {filename}")
        # Предварительная обработка изображения
        processed_img = preprocess_image(img)
        # Извлечение предсказанных цифр
        pred_digits = extract_digits(processed_img)
        #print(f"Распознанные метки: {pred_digits}")
        # Получение истинных меток из аннотаций
        true_digits = [label[0] for label in labels]
        #print(f"Метки: {true_digits}")
        # Запись результатов в файл
        # Формат: filename | распознанные метки | истинные метки
        f.write(f"Файл: {filename}\n")
        f.write(f"Распознанные метки: {''.join(pred_digits)}\n")
        f.write(f"Метки: {''.join(true_digits)}\n")
        f.write("\n")  # разделитель между записями    

        # Преобразование в бинарный формат для вычисления метрик
        true_labels = [1 if str(i) in true_digits else 0 for i in range(10)]
        pred_labels = [1 if str(i) in pred_digits else 0 for i in range(10)]       
        all_true_labels.append(true_labels)
        all_pred_labels.append(pred_labels)


# Вычисление точности
precision = precision_score(all_true_labels, all_pred_labels, average='macro', zero_division=0) 
print(f"Точность: {precision*100:.3f}%")

Точность: 14.444%


In [12]:
def recognize_digits(image_path):
    # Загружаем изображение
    image = cv2.imread(image_path)

    # Преобразуем изображение в оттенки серого
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Применяем пороговое значение для улучшения контраста
    _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)

    # Распознаем текст (цифры) на изображении
    custom_config = r'--oem 3 --psm 6 outputbase digits'
    recognized_text = pytesseract.image_to_string(thresh_image, config=custom_config)

    return recognized_text.strip()

image_name = 'test1.jpg'
result = recognize_digits(image_name)
print(f'Распознанные цифры: {result}')

Распознанные цифры: 7492
404
6


In [13]:
image_name2 = 'test2.jpg'
result2 = recognize_digits(image_name2)
print(f'Распознанные цифры: {result2}')

Распознанные цифры: 01234
16789


In [2]:
#Фонтанное преобразование 2 датасет
import cv2
import numpy as np
import pytesseract
import logging
from sklearn.metrics import precision_score, recall_score, average_precision_score

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Путь к исполняемому файлу Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Tesseract\tesseract.exe"
image_dir = 'Yolo8v2/train/images' # Путь к изображениям
label_dir = 'Yolo8v2/train/labels' # Путь к меткам изображений

def load_images_and_annotations(image_dir, label_dir):
    images = []
    annotations = []
    for filename in os.listdir(image_dir):
        if filename.endswith((".jpg", ".png")):
            # Загрузка изображения
            img_path = os.path.join(image_dir, filename)
            img = cv2.imread(img_path)
            images.append((filename, img))
            
            # Загрузка аннотации
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(label_dir, label_filename)
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    labels = [line.strip().split() for line in f.readlines()]
                annotations.append((filename, labels))
            else:
                annotations.append((filename, []))
    return images, annotations

def preprocess_image(img):
    # Преобразование в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Инвертирование цветов
    inverted = cv2.bitwise_not(gray)
    # Применение адаптивной бинаризации
    thresh = cv2.adaptiveThreshold(inverted, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)
    return thresh

def extract_digits(img):
    # Настройка конфигурации Tesseract для распознавания только цифр
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789'
    result = pytesseract.image_to_string(img, config=custom_config)
    return result.strip()

def calculate_metrics(true_labels, pred_labels):
    precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
    recall = recall_score(true_labels, pred_labels, average='macro', zero_division=0)
    average_precision = average_precision_score(true_labels, pred_labels, average='macro')
    return precision, recall, average_precision

#Загрузка изображений и меток
images, annotations = load_images_and_annotations(image_dir, label_dir)
all_true_labels = []
all_pred_labels = []
with open('results2.txt', 'w', encoding='utf-8') as f:
    for (filename, img), (_, labels) in zip(images, annotations):
        #print(f"Файл: {filename}")
        # Предварительная обработка изображения
        processed_img = preprocess_image(img)
        # Извлечение предсказанных цифр
        pred_digits = extract_digits(processed_img)
        #print(f"Распознанные метки: {pred_digits}")
        # Получение истинных меток из аннотаций
        true_digits = [label[0] for label in labels]
        #print(f"Метки: {true_digits}")
        # Запись результатов в файл
        # Формат: filename | распознанные метки | истинные метки
        f.write(f"Файл: {filename}\n")
        f.write(f"Распознанные метки: {''.join(pred_digits)}\n")
        f.write(f"Метки: {''.join(true_digits)}\n")
        f.write("\n")  # разделитель между записями    

        # Преобразование в бинарный формат для вычисления метрик
        true_labels = [1 if str(i) in true_digits else 0 for i in range(10)]
        pred_labels = [1 if str(i) in pred_digits else 0 for i in range(10)]       
        all_true_labels.append(true_labels)
        all_pred_labels.append(pred_labels)


# Вычисление точности
precision = precision_score(all_true_labels, all_pred_labels, average='macro', zero_division=0) 
print(f"Точность: {precision*100:.3f}%")

Точность: 25.230%


In [ ]:
# YOLO11 неудачно
import os
# Заставляем использовать python-реализацию NMS через переопределение
os.environ["TORCHVISION_NMS_IMPL"] = "python"

import torch
import torchvision

# Сохраняем оригинальную реализацию NMS, которая работает на CPU
_original_nms = torchvision.ops.nms

def custom_nms(boxes, scores, iou_threshold):
    # Переносим данные на CPU, выполняем NMS и возвращаем результат на исходном устройстве
    boxes_cpu = boxes.cpu()
    scores_cpu = scores.cpu()
    keep = _original_nms(boxes_cpu, scores_cpu, iou_threshold)
    return keep.to(boxes.device)

# Переопределяем torchvision.ops.nms на нашу CPU-версию
torchvision.ops.nms = custom_nms

from ultralytics import YOLO


# Определение устройства: 'cuda' для GPU, иначе 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 1. Загрузка предварительно обученной модели YOLO (yolov8x.pt)
model = YOLO('yolo11n.pt')
model.to(device)  # Перевод модели на GPU, если доступен
    
    # 2. Обучение модели на GPU
model.train(data='Numbers_yolov11/data.yaml', epochs=3, imgsz=640, batch=25, name='detection', device=device)
    
    # 3. Оценка модели на GPU
metrics = model.val(device=device)
print(f"mAP для боксов: {metrics.box.map}")
    # print(f"mAP для масок: {metrics.seg.map}") - для сегментации, у меня ошибки выдаёт
    
    # 4. Использование модели для предсказания на тестовых изображениях с использованием GPU
results = model.predict(source='Numbers_yolov11/test/images', save=True, save_txt=True, save_conf=True, device=device)

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=25, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Numbers_yolov11/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=detection2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, s

train: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov11\train\labels.cache... 97 images, 1 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.20.1 ms, read: 55.014.9 MB/s, size: 23.7 KB)



c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov11\valid\labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\detection2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005859375000000001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\detection2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


3 epochs completed in 0.019 hours.
Optimizer stripped from runs\detect\detection2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\detection2\weights\best.pt, 5.5MB

Validating runs\detect\detection2\weights\best.pt...
Ultralytics 8.3.132  Python-3.11.9 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


                   all         23         74    0.00178     0.0578    0.00122   0.000963
                     0         14         14     0.0115     0.0714    0.00757    0.00605
                     1          4          4          0          0          0          0
                     4         10         13          0          0          0          0
                     5         17         28          0          0          0          0
                     6          6          6   0.000974      0.333   0.000984   0.000689
                     7          2          4          0          0          0          0
                     8          5          5          0          0          0          0
Speed: 1.1ms preprocess, 52.0ms inference, 0.0ms loss, 10.4ms postprocess per image
Results saved to runs\detect\detection2
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 364.363.0 MB/s, size: 25.7 KB)


val: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov11\valid\labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


                   all         23         74    0.00178     0.0578    0.00122   0.000963
                     0         14         14     0.0115     0.0714    0.00757    0.00605
                     1          4          4          0          0          0          0
                     4         10         13          0          0          0          0
                     5         17         28          0          0          0          0
                     6          6          6   0.000974      0.333   0.000984   0.000689
                     7          2          4          0          0          0          0
                     8          5          5          0          0          0          0
Speed: 3.4ms preprocess, 47.0ms inference, 0.0ms loss, 11.6ms postprocess per image
Results saved to runs\detect\detection22
mAP для боксов: 0.000963297530517318

image 1/21 c:\PNRPU\PNRPU\    \ 4\Numbers_yolov11\test\images\Video_2024-01-25_142650106_jpg.rf.848e68240ae6864df221714f16ef0

In [ ]:
# YOLO8 1 датасет
import os
import torch
import torchvision
from ultralytics import YOLO

def custom_nms(boxes, scores, iou_threshold):
    boxes_cpu = boxes.cpu()
    scores_cpu = scores.cpu()
    keep = _original_nms(boxes_cpu, scores_cpu, iou_threshold)
    return keep.to(boxes.device)

os.environ["TORCHVISION_NMS_IMPL"] = "python"
_original_nms = torchvision.ops.nms
torchvision.ops.nms = custom_nms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Загрузка предварительно обученной модели YOLO (yolov8x.pt)
model = YOLO('yolov8x.pt')
model.to(device)
    
# Обучение модели 
model.train(data='Numbers_yolov8/data.yaml', epochs=3, imgsz=640, batch=25, name='detection', device=device)
      
# Использование модели для предсказания на тестовых изображениях с использованием GPU
results = model.predict(source='Numbers_yolov8/test/images', save=True, save_txt=True, save_conf=True, device=device)

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=25, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Numbers_yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, sav

train: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov8\train\labels... 97 images, 1 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<00:00, 1030.26it/s]

train: New cache created: C:\PNRPU\PNRPU\    \ 4\Numbers_yolov8\train\labels.cache
val: Fast image access  (ping: 0.10.1 ms, read: 319.3133.9 MB/s, size: 23.7 KB)



c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov8\valid\labels... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 958.12it/s]

val: New cache created: C:\PNRPU\PNRPU\    \ 4\Numbers_yolov8\valid\labels.cache


...


3 epochs completed in 0.658 hours.
Optimizer stripped from runs\detect\detection\weights\last.pt, 136.7MB
Optimizer stripped from runs\detect\detection\weights\best.pt, 136.7MB

Validating runs\detect\detection\weights\best.pt...
Ultralytics 8.3.132  Python-3.11.9 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 112 layers, 68,131,272 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:21<00:00, 21.16s/it]


                   all         23         74      0.309      0.369      0.249      0.177
                     0         14         14       0.23      0.929      0.516      0.397
                     1          4          4     0.0372       0.75       0.52      0.388
                     4         10         13     0.0816      0.154     0.0727     0.0413
                     5         17         28      0.499       0.25      0.351      0.226
                     6          6          6      0.313        0.5      0.265      0.179
                     7          2          4          1          0    0.00223    0.00178
                     8          5          5          0          0     0.0152      0.008
Speed: 1.9ms preprocess, 877.7ms inference, 0.0ms loss, 32.2ms postprocess per image
Results saved to runs\detect\detection
Model summary (fused): 112 layers, 68,131,272 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access  (ping: 0.20.2 ms, read: 274.886.5 MB/s, size: 25.7 KB)


val: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Numbers_yolov8\valid\labels.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:20<00:00, 20.70s/it]


                   all         23         74      0.309      0.369      0.249      0.177
                     0         14         14       0.23      0.929      0.516      0.397
                     1          4          4     0.0372       0.75       0.52      0.388
                     4         10         13     0.0816      0.154     0.0727     0.0413
                     5         17         28      0.499       0.25      0.351      0.226
                     6          6          6      0.313        0.5      0.265      0.179
                     7          2          4          1          0    0.00223    0.00178
                     8          5          5          0          0     0.0152      0.008
Speed: 1.3ms preprocess, 861.7ms inference, 0.0ms loss, 29.4ms postprocess per image
Results saved to runs\detect\detection2
mAP для боксов: 0.17722197884440807

image 1/21 c:\PNRPU\PNRPU\    \ 4\Numbers_yolov8\test\images\Video_2024-01-25_142650106_jpg.rf.848e68240ae6864df221714f16ef0e9

In [ ]:
# YOLO8 2 датасет
import os
import torch
import torchvision
from ultralytics import YOLO

def custom_nms(boxes, scores, iou_threshold):
    boxes_cpu = boxes.cpu()
    scores_cpu = scores.cpu()
    keep = _original_nms(boxes_cpu, scores_cpu, iou_threshold)
    return keep.to(boxes.device)

os.environ["TORCHVISION_NMS_IMPL"] = "python"
_original_nms = torchvision.ops.nms
torchvision.ops.nms = custom_nms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Загрузка предварительно обученной модели YOLO (yolov8x.pt)
model = YOLO('yolov8x.pt')
model.to(device)
    
# Обучение модели 
model.train(data='Yolo8v2/data.yaml', epochs=2, imgsz=640, batch=25, name='detection', device=device)
      
# Использование модели для предсказания на тестовых изображениях с использованием GPU
results = model.predict(source='Yolo8v2/test/images', save=True, save_txt=True, save_conf=True, device=device)

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=25, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Yolo8v2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=detection5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=True

train: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Yolo8v2\train\labels... 309 images, 6 backgrounds, 0 corrupt: 100%|██████████| 309/309 [00:00<00:00, 1683.79it/s]

train: New cache created: C:\PNRPU\PNRPU\    \ 4\Yolo8v2\train\labels.cache
val: Fast image access  (ping: 0.10.0 ms, read: 319.995.1 MB/s, size: 19.6 KB)



c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\PNRPU\PNRPU\Модели и технологии машинного обучения\Лабораторная 4\Yolo8v2\valid\labels... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<00:00, 1983.46it/s]

val: New cache created: C:\PNRPU\PNRPU\    \ 4\Yolo8v2\valid\labels.cache



c:\Users\sergei\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\detection5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005859375000000001), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\detection5
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G      2.749      4.411      2.799        150        640:  54%|█████▍    | 7/13 [21:42<18:45, 187.60s/it]